<h1><center>Bayesian Course</center></h1>

<h2>Lecture 27: Model Assessment and Selection</h2>

<h3>References</h3>

1. Guangyuan Gao, Bayesian Claims Reserving Methods in Non-life Insurance with Stan. An Introduction, Springer, 2018

In [ ]:
%matplotlib inline
import numpy as np
import pystan as ps
import pylab as pl

In [ ]:
schools_code = """
data {
    int<lower=0> J; // number of schools
    real y[J]; // estimated treatment effects
    real<lower=0> sigma[J]; // s.e. of effect estimates
}
parameters {
    real mu;
    real<lower=0> tau;
    real eta[J];
}
transformed parameters {
    real theta[J];
    for (j in 1:J)
    theta[j] <- mu + tau * eta[j];
}
model {
    eta ~ normal(0, 1);
    y ~ normal(theta, sigma);
}
"""

schools_dat = {'J': 8,
               'y': [28,  8, -3,  7, -1,  1, 18, 12],
               'sigma': [15, 10, 16, 11,  9, 11, 10, 18]}

fit = ps.stan(model_code=schools_code, data=schools_dat,
                  iter=1000, chains=4)

In [ ]:
mus = fit.extract()['mu']
pl.hist(mus, color = '#bbbbee')

<h2> 27.1 Introduction</h2>

In this section, we review the model diagnostic tools including posterior predictive
checking and residual plots. We also discuss the model selection criteria including
several information criteria and cross-validation.

<h2> 27.2 Posterior Predictive Checking</h2>

In the Bayesian framework, ideally we want to split the data into a training set and a testing set and do the posterior predictive checking on the testing data set. 


Alternatively, we can choose a test statistic whose predictive distribution does not depend on unknown parameters in the model but primarily on the assumption being checked. 


Then there is no need to have a separate testing data set. Nevertheless, when the same data are used for both fitting and checking the model, this needs to be carried out with caution, as the
procedure can be conservative

<h2> 27.3 Predictive distribution</h2>

The predictive distribution is the distribution of future observations given the
current sample. 

Suppose that $y_{n+1}$ is a future observation
independent of the observed data $\boldsymbol{y}$, given the
underlying $\boldsymbol{\theta}$. Then, the predictive distribution
for $y_{n+1}$ is given by

$$
\begin{equation*}
p(y_{n+1}|\boldsymbol{y}).
\end{equation*}
$$

It can be noted that

$$
\begin{align*}
p(y_{n+1}|\boldsymbol{y})&=\int
p(y_{n+1},\boldsymbol{\theta}|\boldsymbol{y})d\boldsymbol{\theta}\\
&= \int
f(y_{n+1}|\boldsymbol{\theta},\boldsymbol{y})p(\boldsymbol{\theta}|\boldsymbol{y})d\boldsymbol{\theta}\\
&=\int
f(y_{n+1}|\boldsymbol{\theta})p(\boldsymbol{\theta}|\boldsymbol{y})d\boldsymbol{\theta}
\end{align*}
$$

Since

$$
\begin{equation*}
p(y_{n+1}|\boldsymbol{y}) = \int
f(y_{n+1}|\boldsymbol{\theta})p(\boldsymbol{\theta}|\boldsymbol{y})d\boldsymbol{\theta},
\end{equation*}
$$

predictive values can be obtained inside the MCMC algorithm.  Suppose that in the step $m$ after convergence the sample of
$\boldsymbol{\theta}$ is $\boldsymbol{\theta}^{(m)}$. Then, a
predictive value can be drawn from the density

$$
\begin{equation*}
f(y|\boldsymbol{\theta}^{(m)})
\end{equation*}
$$

<h2> 27.4 Residuals, Deviance and Deviance Residuals</h2>

In the Bayesian framework, we can generate a set of residuals for one realization of
posterior parameters. So there are four choices of residuals:
1. Choose the posterior mean of parameters and find one set of residuals.
2. Randomly choose a realization of parameters and find one set of the residuals.
3. Get the posterior mean of residuals.
4. Get the posterior distribution of residuals.

In the following, we will review Pearson residuals, deviance and deviance residuals.
A Pearson residual is defined as


$$
\begin{equation*}
r_i(\theta)= \frac{ y_i -
\mathbb{E}(Y_i|\theta)}{\sqrt{Var(Y_i|\theta)}}, \quad
i=1,\cdots,n
\end{equation*}
$$

The **deviance** is defined as

$$
\begin{equation}
D(\theta) := −2 \log p(y|\theta) = −2\sum_{i=1}^n \log p (yi |\theta) ,
\end{equation}
$$

and the contribution of each data point to the deviance is $D_i(\theta) = −2 \log p (yi |\theta)$. We will define and use $D(\hat{\theta})$ and $\widehat{D(\theta)}$ in the next section.

The deviance residuals are based on a standardized or “saturated” version of the
deviance, defined as

$$
\begin{equation}
D_S(\theta) := −2 \sum_{i=1}^n \log p (y_i |\theta) + 2 \sum_{i=1}^n \log p (yi |\hat{\theta}_S(y_i),
\end{equation}
$$

where $\hat{\theta}_S(y_i)$ are appropriate “saturated” estimates, e.g., we set $\hat{\theta}_S(y)=y$. 

The contribution of each data point to the standardized deviance is

$$
\begin{equation}
D_{S_i}(\theta) := −2  \log p (y_i |\theta) + 2  \log p (yi |\hat{\theta}_S(y_i),
\end{equation}
$$

The deviance residual is defined as

$$
\begin{equation}
dr_i := sign_i \sqrt{D_{S_i}(\theta)},
\end{equation}
$$
where $sign_i$ is the sign of $y_i − \mathbb{E}(y_i|\theta)$.

<h2> 27.5 Three error structures for stack-loss data</h2>

Birkes and Dodge (1993) apply different regression models to the much-analysed stack-loss data of Brownlee (1965). This features 21 daily responses of stack loss $y$, the amount of ammonia escaping, with covariates being air flow ($x_1$) , temperature ($x_2$) and acid concentration ($x_3$). 

We first assume a linear regression on the expectation of $y$, with a variety of different error structures. We assume a linear regression on the expectation of y, i.e.,

$$ \mathbb{E}(y_i) = \mu_i = b_0 + b_1 z_{1i} + b_2 z_{2i} + b_3 z_{3i} $$


   
where $z_{ij} = (x_{ij} - \bar{x}_j ) / \text{sd}(x_j )$ are covariates standardised to have zero mean and unit variance. $b_1 , b_2 , b_3$ are initially given independent "noninformative" priors. 

We consider three error structures: normal, double exponential
and t4, as follows:

$$ 
\begin{align}
y_i &\sim \text{N}(\mu_i , \tau^{-1} )\\
y_i &\sim \text{Double exp}( \mu_i , \tau^{-1} )\\
y_i &\sim \text{t}_4(\mu_i , \tau^{-1} ),
\end{align}
$$

The deviance residuals of the three models have the following forms respectively:

$$ 
\begin{align}
D_{S_i} &= \sqrt{\tau}(y_i-\mu_i)\\
D_{S_i} &= sign_i\sqrt{2\tau|y_i-\mu_i|}\\
D_{S_i} &= sign_i\sqrt{5\log\left( 1 + \frac{(y_i-\mu_i)^2}{4} \right)}.
\end{align}
$$

<h3> 27.5.1 Normal model</h3>


In [ ]:
import numpy as np
stacks_data = {'p': 3,
'N': 21,
'Y': (43, 37, 37, 28, 18, 18, 19, 20, 15, 14, 14, 13, 11, 12, 8, 
7, 8, 8, 9, 15, 15),
'x': np.resize((80, 80, 75, 62, 62, 62, 62, 62, 59, 58, 58, 58, 58, 
58, 50, 50, 50, 50, 50, 56, 70, 27, 27, 25, 24, 22, 23, 24, 24, 
23, 18, 18, 17, 18, 19, 18, 18, 19, 19, 20, 20, 20, 89, 88, 90, 
87, 87, 87, 93, 93, 87, 80, 89, 88, 82, 93, 89, 86, 72, 79, 80, 
82, 91), (21, 3))}

In [ ]:
import pystan
%load_ext stanmagic
%matplotlib inline
import numpy as np
import pylab as pl

In [ ]:
%%stan -f normal_stack_loss_reg.stan
// Linear Model with normal Errors

data {
  int<lower=0> N;  // number of observations
  int<lower=0> p;  // number of explained variables
  real Y[N];       // observations
  matrix[N,p] x;   // design matrix
} 

// to standardize the x's 
transformed data {
  real z[N,p];
  real mean_x[p];
  real sd_x[p];
  for (j in 1:p) { 
    mean_x[j] <- mean(col(x,j)); 
    sd_x[j] <- sd(col(x,j)); 
    for (i in 1:N)  
      z[i,j] <- (x[i,j] - mean_x[j]) / sd_x[j]; 
  }
}

parameters {
  real beta0; 
  real beta[p]; 
  real<lower=0> sigma; 
} 

transformed parameters {
  real<lower=0> tau;
  real mu[N];
  tau <-1/sigma;
  for (n in 1:N)
    mu[n] <- beta0 + beta[1] * z[n, 1] + beta[2] * z[n, 2] + beta[3] * z[n, 3];
}

model {
  beta0 ~ normal(0, 5); 
  beta ~ normal(0, 5);
  sigma ~ cauchy(0, 2);
  for (n in 1:N) 
    //Y[n] ~ double_exponential(mu[n], sigma); //try you
    //Y[n] ~ student_t(4, mu[n], sigma); //try you
    Y[n] ~ normal(mu[n], sigma); 
} 

generated quantities {
  real b0;
  real b[p];
  real residual[N]; // Pearson residual
  real outlier[N];  // test for outlier
  real resi_dev[N]; // deviance 
  real y_rep[N];
  real pval:

  for (j in 1:p)
    b[j] <- beta[j] / sd_x[j];
  b0 <- beta0 - b[1] * mean_x[1] - b[2] * mean_x[2] - b[3] * mean_x[3];

  for (i in 1:N){
    residual[i] = (Y[i] - mu[i]) / sigma;
    outlier[i] = step(fabs((Y[i] - mu[i]) / sigma) - 2.5);
    resi_dev[i] = \sqrt(tau)*(Y[i] - mu[i]);
    y_rep[i] = normal_rng(mu[i],sigma);
 }
 
    pval = step(sum(normal_lpdf(Y,mu,sigma))-sum(normal_lpdf(y_rep,mu,sigma)));
}    

In [ ]:
_stan_model

In [ ]:
normal_stack_loss_reg = pystan.StanModel(file=_stan_model.model_file)

In [ ]:
normal_stack_loss_reg

In [ ]:
normal_stack_loss_reg_sample = normal_stack_loss_reg.sampling(data=stacks_data)

In [ ]:
normal_stack_loss_reg_sample

In [ ]:
traces = normal_stack_loss_reg_sample.extract(permuted=True)

In [ ]:
head(traces)

<h4> Some other models for the data to explore.</h4>

Please review

In [ ]:
stacks_lasso = '''
data {
  int<lower=0> N;
  int<lower=0> p;
  real Y[N];
  matrix[N,p] x;
} 

// to standardize the x's 
transformed data {
  real z[N,p];
  real mean_x[p];
  real sd_x[p];
  for (j in 1:p) { 
    mean_x[j] <- mean(col(x,j)); 
    sd_x[j] <- sd(col(x,j)); 
    for (i in 1:N)  
      z[i,j] <- (x[i,j] - mean_x[j]) / sd_x[j]; 
  }
}

parameters {
  real beta0; 
  real beta[p]; 
  real<lower=0> sigmasq; 
  real<lower=0> phi;
} 

transformed parameters {
  real<lower=0> sigma;
  real mu[N];

  sigma <- sqrt(2) * sigmasq;
  for (n in 1:N)
    mu[n] <- beta0 + beta[1] * z[n, 1] + beta[2] * z[n, 2] + beta[3] * z[n, 3];
}

model {
  beta0 ~ normal(0, 316); 
  phi ~ gamma(0.01, 0.01);
  beta ~ normal(0, sqrt(phi));
  sigmasq ~ inv_gamma(.001, .001); 
  for (n in 1:N) 
    Y[n] ~ double_exponential(mu[n], sigmasq); 
} 

generated quantities {
  real b0;
  real b[p];
  real outlier_1;
  real outlier_3;
  real outlier_4;
  real outlier_21;

  for (j in 1:p)
    b[j] <- beta[j] / sd_x[j];
  b0 <- beta0 - b[1] * mean_x[1] - b[2] * mean_x[2] - b[3] * mean_x[3];

  outlier_1  <- step(fabs((Y[1] - mu[1]) / sigma) - 2.5);
  outlier_3  <- step(fabs((Y[3] - mu[3]) / sigma) - 2.5);
  outlier_4  <- step(fabs((Y[4] - mu[4]) / sigma) - 2.5);
  outlier_21 <- step(fabs((Y[21] - mu[21]) / sigma) - 2.5);
}
'''

In [ ]:
stacks_init = {'beta0': 10,
'beta': [0, 0, 0],
'sigmasq': 10,
'phi': 0.316}

In [ ]:
fit_lasso = ps.stan(model_code=stacks_lasso, data=stacks_data, 
                  iter=1000, chains=4)

In [ ]:
import seaborn as sb

traces = fit_lasso.extract(permuted=True)

In [ ]:
import pandas as pd

betas = pd.DataFrame(traces['beta'], columns=['beta1','beta2','beta3']).stack().reset_index()
betas.columns = 'obs', 'parameter', 'value'
g = sb.FacetGrid(betas, col='parameter')
g.map(pl.hist, 'value', color="steelblue",lw=0)

In [ ]:
stacks_ridge = '''
data {
  int<lower=0> N;
  int<lower=0> p;
  real Y[N];
  matrix[N,p] x;
} 

// to standardize the x's 
transformed data {
  matrix[N,p] z;
  row_vector[p] mean_x;
  real sd_x[p];
  real d;

  for (j in 1:p) { 
    mean_x[j] <- mean(col(x,j)); 
    sd_x[j] <- sd(col(x,j)); 
    for (i in 1:N)  
      z[i,j] <- (x[i,j] - mean_x[j]) / sd_x[j]; 
  }
  d <- 4; // degrees of freedom for t
}

parameters {
  real beta0; 
  vector[p] beta;
  real<lower=0> sigmasq; 
  real<lower=0> phi;
} 

transformed parameters {
  real<lower=0> sigma;
  vector[N] mu;

  sigma <- sqrt(d * sigmasq / (d-2)); // t errors on d degrees of freedom
  mu <- beta0 + z * beta; 
}

model {
  beta0 ~ normal(0, 316); 
  phi ~ gamma(0.01, 0.01);
  beta ~ normal(0, sqrt(phi));
  sigmasq ~ inv_gamma(.001, .001); 
  Y ~ student_t(d, mu, sqrt(sigmasq));
} 

generated quantities {
  real b0;
  vector[p] b;
  real outlier_1;
  real outlier_3;
  real outlier_4;
  real outlier_21;

  for (j in 1:p)
    b[j] <- beta[j] / sd_x[j];
  b0 <- beta0 - mean_x * b;

  outlier_1  <- step(fabs((Y[3] - mu[3]) / sigma) - 2.5);
  outlier_3  <- step(fabs((Y[3] - mu[3]) / sigma) - 2.5);
  outlier_4  <- step(fabs((Y[4] - mu[4]) / sigma) - 2.5);
  outlier_21 <- step(fabs((Y[21] - mu[21]) / sigma) - 2.5);
}
'''

In [ ]:
fit_ridge = ps.stan(model_code=stacks_ridge, data=stacks_data, init=stacks_init,
                  iter=10000, chains=4)

In [ ]:
ridge_traces = fit_ridge.extract(permuted=True)

ridge_betas = pd.DataFrame(ridge_traces['beta'], columns=['beta1','beta2','beta3']).stack().reset_index()
ridge_betas.columns = 'obs', 'parameter', 'value'
g = sb.FacetGrid(ridge_betas, col='parameter')
g.map(pl.hist, 'value', color="steelblue",lw=0)

In [ ]:
pl.plot(ridge_traces['beta'])

In [ ]:
basket_code = '''
data {
    int<lower=0> K;
    int<lower=0> y[K];
    int<lower=0> N[K];
}
parameters {
    real theta[K];
    real mu;
    real<lower=0> s2;
}
transformed parameters {
    real<lower=0, upper=1> p[K];
    for (i in 1:K)
        p[i] <- inv_logit(theta[i])
}
model {
    mu ~ normal(-1.34, 100)
    s2 ~ inv_gamma(0.00005, 0.000005)
    for (i in 1:K)
        theta[i] ~ normal(mu, sqrt(s2))
        y[i] ~ binomial(n[i], p[i])
}

'''

In [ ]:
basket_indep_code = '''
data {
    int<lower=0> K;
    int<lower=0> y[K];
    int<lower=0> N[K];
}
parameters {
    real theta[K];
}
transformed parameters {
    real<lower=0, upper=1> p[K];
    for (i in 1:K)
        p[i] <- inv_logit(theta[i])
}
model {
    for (i in 1:K)
        theta[i] ~ normal(-1.34, 100)
        y[i] ~ binomial(n[i], p[i])
}

'''

<h2> Bayesian Model selection</h2>

The model selection problem is a trade-off between a simple model and good fitting.

Ideally, we want to choose the simplest model with best fitting. 

However good fitting models tend to be more complicated while simpler models tend to be underfit.

The model selection methods used in frequentist statistics are typically cross-validation
and information criteria, which are the modified residual sum of squares with respect
to the model complexity and overfitting.


Cross-validation measures the fit of a model on the testing data set, which is not
used to fit the model, while the information criteria adjust the measure of fit on the
training data set by adding a penalty for model complexity.


<h3>The Predictive Accuracy of a Model</h3>

In the Bayesian framework, the fit of a model is sometimes called the predictive
accuracy of a model (Gelman et al. 2014). We measure the predictive accuracy of
a model to a data set $y'$ by log point wise predictive density (**lppd**), calculated as
follows:

$$
lppd:= \log \prod_{i=1}^{n'} \mathbb{E}_{\theta|y} p(y'|\theta) = \sum_{i=1}^{n'} \log (\mathbb{E}_{\theta|y} p(y'|\theta)) = \sum_{i=1}^{n'} \log \left( \int p(y'|\theta) p(\theta|y)d\theta \right).
$$

Ideally, $y'$ should not be used to fit the model. If we choose $y' = y$, we get the
within-sample lppd (denoted by $lppd_{train}$), which is typically larger than the out-ofsample
lppd (denoted by $lppd_{test}$). 


To compute **lppd** in practice, we can evaluate the expectation using draws from the posterior distribution $p(θ|y)$, which we label as $\theta^{(t)}, t = 1, \ldots, T$ . The computed lppd is defined as follows:
 

$$
\text{computed } lppd:= \sum_{i=1}^{n} \log\left( \frac{1}{T} \sum_{t=1}^{T} p(y'|\theta^{(t)})\right).
$$

<h3>Cross Validation</h3>

In Bayesian cross-validation, the data are repeatedly partitioned into a training set
$y_{train}$ and a testing set $y_{test}$. 


For simplicity, we restrict our attention to leave-one-out cross-validation (LOO-CV), where ytest only contains a data point. The Bayesian LOO-CV estimate of out-of-sample lppd is defined as follows:

$$
lppd_{loo-cv} := \sum_{i=1}^{n} \log \left( p (y_i |\theta) p (\theta|y_{−i} ) d\theta\right).
$$

where $y_{−i}$ is the data set without the $i$-th point. The $lppd_{loo-cv}$ can be computed as

$$
\text{computed } lppd_{loo-cv} = \sum_{i=1}^{n} \log  \left( \sum_{t=1}^{T} p(y'|\theta^{(it)}) \right),
$$

where $\theta^{(it)}, t = 1, \ldots , T$ are the simulations from the posterior distribution $p(\theta|y_{-i})$.

<h3>Deviance Information Criterion DIC</h3>

Before describing the DIC, we review another two information criteria employed in
frequentist statistics. The Akaike information criterion (AIC) by Akaike (1973) is
defined as

$$
AIC:= − 2\sum_{i=1}^{n} \log p (y_i |\theta_{MLE}) + 2p.
$$


The Bayesian information criterion (BIC) by Schwarz (1978) is defined as


$$
AIC:= − 2\sum_{i=1}^{n} \log p (y_i |\theta_{MLE}) + p\log n
$$,

where $p$ is the number of parameters. The first common term $− 2\sum_{i=1}^{n} \log p (y_i |\theta_{MLE})$
measures the discrepancy between the fitted model and the data. The second term measures the model complexity.


<h4> DIC </h4>

In the Bayesian framework, we define a similar quantity to measure the discrepancy,
$-2 \sum_{i=1}^{n} \log p(y_i | \hat{\theta})$, where $\hat{\theta}$ is the posterior mean. Spiegelhalter et al. (2002) proposed a measure of number of effective parameters, which is defined as the difference
between the posterior mean of the deviance and the deviance at the posterior means,
as follows:

$$
p_D := \widehat{D (\theta)} - D(\hat{\theta}) = −2\mathbb{E}_{\theta|y} \left(\sum_{i=1}^{n} \log p (y_i |\theta)\right) + 2 \sum_{i=1}^{n} \log p(y_i|\hat{\theta}),
$$)

where $D$ is the deviance defined above.


Furthermore, they proposed a deviance information criterion (DIC), defined as
the deviance at the posterior means plus twice the effective number of parameters,
to give

$$
DIC := D( \hat{\theta}) + 2p_D.
$$

DIC is viewed as a Bayesian analogue of AIC. We prefer the model with smaller
DIC. 
Note that DIC and $p_D$ are sensitive to the level of a hierarchical model. They are appropriate when we are interested in the parameters directly related to the data.


<h3> Watanabe-Akaike or widely available information criterion (WAIC)</h3>

Watanabe (2010) proposed another measure of number of effective parameters as follows:

$$
p_{WAIC} := \widehat{D(\theta)} + 2lppd_{train} = −2\mathbb{E}_{\theta|y} \left(\sum_{i=1}^{n} \log p (y_i |\theta) \right) + 2 \sum_{i=1} ^{n} \log \left( \mathbb{E}_{\theta|y} p (y_i |\theta)\right),
$$

where $−2lppd_{train}$ plays a role as $D(\hat{\theta})$ as in $p_D$. As with AIC and DIC, the Watanabe-
Akaike information criterion (WAIC) is defined as

$$
WAIC:= − 2lppd_{train} + 2p_{WAIC}.
$$

<h3> Leave-One-Out Information Criterion (LOOIC)</h3>

Different from the definition of number of effective parameters in AIC, DIC, and
WAIC, we define

$$
p_{loo} := lppd_{train} − lppd_{loo-cv},
$$

where $lppd_{loo-cv}$ comes from above. The leave-one-out information criterion (LOOIC) is defined as

$$
LOOIC:= − 2lppd_{train} + 2p_{loo} = −2lppd_{loo-cv},
$$

which is reasonable since $lppd_{loo-cv}$ already penalizes the overfitting (or equivalently
the model complexity).